In [ ]:
# ------------------------------------------------
# MessagesPlaceholder: đóng vai trò như bộ nhớ, duy trì input của người dùng cho generator
# ------------------------------------------------

import operator
from typing import TypedDict, List, Sequence, Annotated
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import MessageGraph, StateGraph, END

# Tạo prompt cho generate và reflection
generation_prompt = ChatPromptTemplate.from_messages([
    ("system", "..."),
    MessagesPlaceholder(variable_name="messages"),
])

reflection_prompt = ...  # TODO: định nghĩa prompt reflection

# Dùng toán tử pipe (|) để kết hợp với LLM
generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

# ------------------------------------------------
# MessageGraph: graph đặc biệt với state là mảng tin nhắn (HumanMessage, AIMessage, SystemMessage, ToolMessage)
# ------------------------------------------------

graph = MessageGraph()

# Định nghĩa state
class MessagesState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add] # We now track a list of messages

graph = StateGraph(MessagesState)

# ------------------------------------------------
# Xây dựng các node
# ------------------------------------------------

def generation_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    generated_tweet = generate_chain.invoke({"messages": state})
    # Khi node trả về AIMessage, LangGraph sẽ tự merge state bằng cơ chế tối ưu
    return [AIMessage(content=generated_tweet.content)]

def reflection_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": state})
    return {"messages":[HumanMessage(content=res.content)]}

graph.add_node("generate", generation_node)
graph.add_node("reflect", reflection_node)

graph.add_edge("reflect", "generate")
graph.set_entry_point("generate")

# ------------------------------------------------
# Router node để kiểm soát luồng
# ------------------------------------------------

def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return "reflect"

graph.add_conditional_edges("generate", should_continue)
